# **Preprocesamiento de datos**
---
> **Asignatura:** Minería de Datos
>
> **Alumna:** Claudia Luz Rojas Soto
>
> **Código:** 171805



## **Algoritmos:**
---

### 1. Distancia de Minkowski (Dist. Euclidiana - Dist. Manhattan)
--- 

In [1]:
import numpy as np
def minkowski(p,x,y):
    """Esta función calcula la distancia de Minkowski de dos arreglos.

    Args:
        p (int): Un nḿero entero (potencia).
        x ,y (array): Arreglos de donde se obtiene distancia

    Returns:
        float: Valor de la distancia de minkowski.
    """
    suma = 0
    for i in range(len(x)):
        if (len(x) == len(y)):
            DistPotencia = np.power(np.abs(x[i]-y[i]),p)
            suma = suma + DistPotencia
    return np.power(suma,1/float(p))

In [2]:
# RDD a partir de una arreglo de arreglos
arreglos = sc.parallelize([[[1 ,2 ,3 ,4], [10, 9, 8, 7]],[[3, 4, 5, 6], [4, 5, 6, 7]]])

# Distancia de Manhattan
Manhattan = arreglos.map(lambda x: minkowski(1, x[0], x[1]))
# Mostrar la distancia de Manhattan obtenida
print('Distancias de Manhattan:',Manhattan.collect())

# Distancia Euclidiana 
Euclidiana = arreglos.map(lambda x: minkowski(2, x[0], x[1]))
# Mostrar la distancia Euclidiana obtenida
print('Distancias euclidianas:',Euclidiana.collect())


Distancias de Manhattan: [24.0, 4.0]
Distancias euclidianas: [12.806248474865697, 2.0]


### 2. Similitud de Cosenos
--- 

In [3]:
def sim_cosenos(x,y):
    """Calcula la distancia de cosenos

    Args:
        x, y (num array): Listas de valores de las cuales se calculara similitud

    Returns:
        double: Un número de tipo double equivalente a la distancia de cosenoS de x e y
    """
    # Lista de tuplas con la forma (x,y)
    tuples = x.zip(y)
    # Obtenemos la norma de x y y de los pares, y luego sumamos(producto punto)
    normx = tuples.map(lambda x: x[0]*x[0]).sum()
    normy = tuples.map(lambda x: x[1]*x[1]).sum()
    # Obtenemos el producto de x y y de los pares, y luego sumamos(producto punto)
    producto = tuples.map(lambda x: x[0]*x[1]).sum()
    # Obtenemos el resultado, dada la fórmula de similitud de cosenos
    similitud = producto/(normx*normy)
    # Devolvemos el valor de la similitud
    return similitud

In [4]:
# Declaramos dos arreglos de la misma dimensión
x = [25,10,56,120,23,5,3,2,32,21,132,123,231,12,54,2]
y = [15,12,56,8,6,9,2,3,3,3,21,132,123,12,54,55]
# Volvemos estos arreglos RDDs
x = sc.parallelize(x,2)
y = sc.parallelize(y,2)

In [5]:
print(sim_cosenos(x,y))

1.187589912259901e-05


### 3. Distancia de Jaccard
--- 

In [6]:
def Jaccard(x,y): 
    """Calcula la distancia de Jaccarda entre dos vectores binarios.

    Args:
        x, y (np.array): Arreglo de enteros binarios x y y.

    Returns:
        J (int): Distancia de Jaccard entre x y y
    """
    return (x==y).sum()/float( np.maximum(x,y).sum() )

In [7]:
# RDD de textos tokenizados para prueba
tokenizado = sc.parallelize(enumerate([['cusco', 'lima', 'cusco','perú'],
                             ['destino', 'viaje', 'cusco','lima']]))

In [8]:
palabras_rep = (tokenizado
             .flatMap(lambda y: y[1])
             .map(lambda x: (x,1))
             .reduceByKey(lambda x,y: x + y)
             )
frecuencia = dict((k,v) for k,v in palabras_rep.collect())
n_palabras= len(frecuencia)
print (frecuencia, n_palabras)

{'lima': 2, 'perú': 1, 'destino': 1, 'cusco': 3, 'viaje': 1} 5


In [9]:
def Binarizacion(atributos,palabras):  
    """Binarizacion de vectores usando un diccionario de claves.

    Args:
        atributos (list): Lista de atributos de un objeto dado.
        palabras (dict): Diccionario de la forma atributo -> index

    Returns:
        array (np.array): Arreglo binario de atributos.
    """
    
    array = np.zeros(len(palabras))
    for atr in atributos:
        array[ list(palabras.keys()).index(atr) ] = 1
    return array

In [10]:
binarizado = tokenizado.map(lambda x: (x[0],Binarizacion(x[1], frecuencia)))
binarizado.collect()

[(0, array([1., 1., 0., 1., 0.])), (1, array([1., 0., 1., 1., 1.]))]

In [11]:
# Realizamos el producto cartesiano de los arreglos
prodCart = binarizado.cartesian(binarizado)

# Mapeamos para transformar el RDD en un RDD de tuplas ((id1,id2), (vector1,vector2))
BinPares = prodCart.map(lambda x: ((x[0][0],x[1][0]),(x[0][1],x[1][1])))

# Mapeamos el RDD de tuplas para calcular la distancia de Jaccard entre los pares
jaccard = BinPares.map(lambda x: Jaccard(x[1][0],x[1][1]))

# Encontramos la distancia máxima, mínima y media de los valores del RDD.
Jmin, Jmax, Jmean = jaccard.min(), jaccard.max(), jaccard.mean()

# Mostramos los resultados
print ("Jaccard:\nmin:{:.2f}\tmax:{:.2f}\tmedia:{:.2f}".format(Jmin, Jmax, Jmean ))

Jaccard:
min:0.40	max:1.67	media:0.93


### 4. Estandarización
---

In [12]:
def estandarizar(value,media,des):
    """Estandariza el valor de un número dentro de un arreglo.

    Args:
        value (int): Valor a escalar.
        media (float):  Media aritmética de los valores del arreglo donde esta el value.
        des (float): Desviación estándar del arreglo donde esta el value.

    Returns:
        float: Valor estandarizado.
    """
    return round(((value - media)/des),3)

In [13]:
# Creamos un arreglo de valores que estandarizaremos
stand = [10,20,2,3,4,5,5,1,2,3,5,7,8,11,34,3]
# Volvemos este arreglo en un RDD, con 4 particiones
standRDD = sc.parallelize(stand,4)
# Obtenemos la media del arreglo
mean = standRDD.mean()
# Obtenemos la varianza del arreglo
var = standRDD.map(lambda x: (x - mean)**2).mean()
# Obtenemos la desviación estándar del arreglo
dsv = var ** 0.5
# Escalamos los valores del arreglo llamando a la función "escalar"
stand_ = standRDD.map(lambda x: (x-mean - mean)/dsv)
# Mostramos el arreglo con un escalonamiento de sus valores
print(stand_.collect())

[-0.656833112811402, 0.5651819807912064, -1.6344451876934887, -1.5122436783332278, -1.390042168972967, -1.267840659612706, -1.267840659612706, -1.7566466970537495, -1.6344451876934887, -1.5122436783332278, -1.267840659612706, -1.0234376408921844, -0.9012361315319236, -0.5346316034511411, 2.276003111834858, -1.5122436783332278]


### 5. Escalonamiento
---

In [14]:
def escalar(value,minimo,maximo):
    """Escala el valor de un número dentro de un arreglo.

    Args:
        value (int): Valor a escalar.
        minimo (float): Mínimo valor del arreglo donde esta el value.
        maximo (float): Máximo valor del arreglo donde esta el value.

    Returns:
        float: Valor escalonado.
    """
    return round((value - minimo) / (maximo-minimo),3)

In [15]:
# Creamos un arreglo de valores que escalonaremos
scaleBs = [10,20,2,3,4,5,5,1,2,3,5,7,8,11]
# Volvemos este arreglo en un RDD, con 7 particiones
scaleRDD = sc.parallelize(scaleBs,7)
# Recogemos el valor mínimo y máximo del arreglo
min_,max_= scaleRDD.min(),scaleRDD.max()
# Escalamos los valores del arreglo llamando a la función "escalar"
scale = scaleRDD.map(lambda x: escalar(x,min_,max_))
# Mostramos el arreglo con un escalonamiento de sus valores
print(scale.collect())

[0.474, 1.0, 0.053, 0.105, 0.158, 0.211, 0.211, 0.0, 0.053, 0.105, 0.211, 0.316, 0.368, 0.526]


### 6. Normalización
---

In [16]:
def normalizar(value,normal):
    """Normaliza el valor de un número dentro de un arreglo.

    Args:
        value (int): Valor a normalizar.
        normal (float): Normal del arreglo donde esta el value.

    Returns:
        float: Valor normalizado.
    """
    return round(value/normal,3)

In [17]:
# Creamos un arreglo de valores de normalizaremos
normTest = [3,2,4,120,23,5,3,2,32,21,132,123,231,12,54]
# Volvemos este arreglo en un RDD con 3 particiones
normRDD = sc.parallelize(normTest,3)
# Obtenemos la norma del vector
nx = (normRDD.map(lambda x: x**2).sum())**0.5
# Normalizamos los valores de arreglo usando map
normal = normRDD.map(lambda x: normalizar(x,nx))
# Mostramos el arreglo con una normalización de sus valores
print(normal.collect())

[0.009, 0.006, 0.012, 0.37, 0.071, 0.015, 0.009, 0.006, 0.099, 0.065, 0.407, 0.379, 0.711, 0.037, 0.166]


### 7. Binarizado
---

In [18]:
def Binarizado(lista):
    """Binariza los elementos de una lista de strings
    Args: 
        lista : arreglo de strings
    Returns: 
        [(string,1)]: vector de tuplas
    """
    return list(map(lambda x:(x,1),lista))

In [19]:
tokenizado = sc.parallelize([['cusco', 'lima', 'cusco','perú'],
                             ['destino', 'viaje', 'cusco','lima']])
#Aplicamos la binarización a los arreglos de strings
binario = tokenizado.map(lambda x: Binarizado(x))


In [20]:
binario.collect()

[[('cusco', 1), ('lima', 1), ('cusco', 1), ('perú', 1)],
 [('destino', 1), ('viaje', 1), ('cusco', 1), ('lima', 1)]]